# 스타벅스 매장 데이터와 지도


* json_normalize(): JSON 데이터 DataFrame으로 전환
* folium 을 사용하여 지도에 위치 표시

# 스타벅스 매장찾기
http://www.istarbucks.co.kr

* STORE >> 매장 찾기
* 지역검색 >> 서울 >> 전체

In [22]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
%config InlineBackend.figure_format = 'retina'

In [23]:
data = {
    'ins_lat':'37.56682', # 지정한 위도와 경도에서 가까운 순으로 나열
    'ins_lng':'126.97865',
    'p_sido_cd':'01', # 01=서울시, 08=경기 ... 16=제주
    'p_gugun_cd':'',  # 세부지역 (지정하지 않으면 시/도 전체)
    'in_biz_cd':'',
    'set_date':'',
    'iend':'1000',
}
    
url = 'https://www.istarbucks.co.kr/store/getStore.do'
r = requests.post(url, data=data)

r.text[:1000] # 수신된 데이터의 앞부분만 확인

'{"list":[{"seq":0,"sido_cd":null,"sido_nm":null,"gugun_cd":null,"gugun_nm":null,"code_order":null,"view_yn":null,"store_num":null,"sido":null,"gugun":null,"address":null,"new_img_nm":null,"p_pro_seq":0,"p_view_yn":null,"p_sido_cd":"","p_gugun_cd":"","p_store_nm":null,"p_theme_cd":null,"p_wireless_yn":null,"p_smoking_yn":null,"p_book_yn":null,"p_music_yn":null,"p_terrace_yn":null,"p_table_yn":null,"p_takeout_yn":null,"p_parking_yn":null,"p_dollar_assent":null,"p_card_recharge":null,"p_subway_yn":null,"stb_store_file_renew":null,"stb_store_theme_renew":null,"stb_store_time_renew":null,"stb_store_lsm":null,"s_code":"1434","s_name":"논현역사거리","tel":"02-3442-3672","fax":"02-3442-3673","sido_code":"01","sido_name":"서울","gugun_code":"0101","gugun_name":"강남구","addr":"서울특별시 강남구 논현동 142-2 정일빌딩","park_info":null,"new_state":null,"theme_state":"T04@T05@T08@T16@T17@T20@P70@P80","new_bool":0,"search_text":"","ins_lat":"","ins_lng":"","in_distance":0,"out_distance":"7.38","all_search_cnt":-1,"addr_sea

데이터는 JSON으로 수신되며, 형태는 다음과 같다.

```json
{
  "list": [
      {
            // ... 중략 ...
            "s_code": "1311",
            "s_name": "방화DT",
            "tel": "02-2664-3480",
            "fax": "02-2664-3481",
            "sido_code": "01",
            "sido_name": "서울",
            "gugun_code": "0103",
            "gugun_name": "강서구",
            "addr": "서울특별시 강서구 방화동 293-4",
            "park_info": null,
            "new_state": null,
            "theme_state": "T17@T16@T09@T20@T01@T05@T08@T04",
            // ... 중략 ...
            "lat": "37.574339",
            "lot": "126.816415",
            "t22": 0
        },
        {
            // ... 중략 ...
            "s_code": "1267",
            "s_name": "마곡나루역",
            "tel": "02-3662-3504",
            "fax": "02-3662-3505",
            "sido_code": "01",
            "sido_name": "서울",
            "gugun_code": "0103",
            "gugun_name": "강서구",
            "addr": "서울특별시 강서구 마곡동 759-3 보타닉파크타워Ⅰ105,203,204호",
            "park_info": null,
            "new_state": null,
            "theme_state": "T08@T05@T04@T17@T16@P80@T20",
            // ... 중략 ...
            "lat": "37.56813",
            "lot": "126.82614",
            "t22": 0
        },
    ]
 }
```

# JSON to DataFrame

json_normalize() 를 사용하여 JSON 데이터를 DataFrame로 전환 

In [24]:
jo = json.loads(r.text)
df = json_normalize(jo, 'list')

In [25]:
# 행(row)수, 서울 457개 매장
len(df) 

486

In [26]:
# 컬럼수 111개
df.columns

Index(['addr', 'addr_search_cnt', 'address', 'all_search_cnt', 'code_order',
       'cold_blew', 'defaultimage', 'disp', 'doro_address', 'espresso',
       ...
       't10', 't12', 't20', 't21', 't22', 'tel', 'theme_state', 'vSal',
       'view_yn', 'whcroad_yn'],
      dtype='object', length=117)

In [27]:
# 주요한 컬럼 몇 가지 선택
               
df = df[['s_name', 'lat', 'lot', 'sido_name', 'gugun_name', 'doro_address', 'tel']]
df.head(20)

,s_name,lat,lot,sido_name,gugun_name,doro_address,tel
0,논현역사거리,37.510178,127.022223,서울,강남구,서울특별시 강남구 강남대로 538 (논현동),02-3442-3672
1,대치대원빌딩R,37.494668,127.062583,서울,강남구,서울특별시 강남구 남부순환로 2947 (대치동),02-568-3704
2,삼성역섬유센터R,37.507750,127.060651,서울,강남구,서울특별시 강남구 테헤란로 518 (대치동),02-568-3648
3,압구정R,37.5273669,127.033061,서울,강남구,서울특별시 강남구 언주로 861 (신사동),02-541-3622
4,수서역R,37.488008,127.102650,서울,강남구,서울특별시 강남구 광평로 281 (수서동),02-6181-3688
5,양재강남빌딩R,37.485192,127.036685,서울,강남구,서울특별시 강남구 남부순환로 2621 (도곡동),02-571-3604
6,선릉동신빌딩R,37.505321,127.050409,서울,강남구,서울특별시 강남구 테헤란로 409 (삼성동),02-2051-3562
7,봉은사로선정릉,37.511293,127.048409,서울,강남구,서울특별시 강남구 봉은사로 446 (삼성동),02-539-3564
8,강남오거리,37.502117,127.026672,서울,강남구,서울특별시 강남구 봉은사로2길 39 (역삼동),02-557-3484
9,스타필드코엑스몰R,37.50999,127.061455,서울,강남구,서울특별시 강남구 영동대로 513 (삼성동),02-6002-3467


# 위도 경도 데이터 타입 변환

In [28]:
df.dtypes

s_name          object
lat             object
lot             object
sido_name       object
gugun_name      object
doro_address    object
tel             object
dtype: object

### lat, lot타입으로 전환 (str→float) 

In [29]:
df['lat'] = df['lat'].astype(float)
df['lot'] = df['lot'].astype(float)

In [30]:
df.dtypes

s_name           object
lat             float64
lot             float64
sido_name        object
gugun_name       object
doro_address     object
tel              object
dtype: object

### 특정 지점

In [31]:
df[df['s_name'] == '시청'] # 시청점

,s_name,lat,lot,sido_name,gugun_name,doro_address,tel
418,시청,37.56629,126.979808,서울,중구,"서울특별시 중구 을지로 19, 삼성화재삼성빌딩 1층 (을지로1가)",02-758-8504


스타벅스 시청점의 위도(lat)와 경도(lot)는 각각  37.56629, 126.979808

# folium
https://github.com/python-visualization/folium

파이썬 지리정보 시각화 모듈  (문서: https://folium.readthedocs.io )

In [32]:
!pip install folium

You are using pip version 18.0, however version 19.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [33]:
import folium

# 서울 시청 
map_osm = folium.Map(location=(37.56629, 126.979808))
map_osm

지도를 상하좌우로 움직이거나 확대/축소 할 수 있다

In [34]:
# 확대 지정 (zoom_start)
map_osm = folium.Map(location=(37.56629, 126.979808), zoom_start=17)
map_osm

In [35]:
# 지도 tiles 지정
map_osm = folium.Map(location=(37.56629, 126.979808), zoom_start=17, tiles='Stamen Toner')
map_osm

In [36]:
시청_좌표=(37.56629, 126.979808)
 
map_osm = folium.Map(location=시청_좌표, zoom_start=17)
folium.Marker(시청_좌표, popup='시청').add_to(map_osm)
map_osm

# 서울 스타벅스 전 지점

In [37]:
map_osm = folium.Map(location=시청_좌표, zoom_start=11)

for ix, row in df.iterrows():
    location = (row['lat'], row['lot'])
    folium.Marker(location, popup=row['s_name']).add_to(map_osm)

map_osm

# 제주도 스타벅스 매장 지도

In [38]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize

한라산_좌표 = (33.361666, 126.529166)

data = {
    'ins_lat': '33.4996213', # 제주 시청의 좌표
    'ins_lng': '126.5311884',
    'p_sido_cd':'16', # 01=서울시, 08=경기 ... 16=제주
    'p_gugun_cd':'',  # 세부지역 (지정하지 않으면 시/도 전체)
    'in_biz_cd':'',
    'set_date':'',
    'iend':'1000',
}
    
url = 'https://www.istarbucks.co.kr/store/getStore.do'
r = requests.post(url, data=data)
df = json_normalize(json.loads(r.text), 'list')

map_osm = folium.Map(location=(33.361666, 126.529166), zoom_start=10)
for ix, row in df.iterrows():
    location = (row['lat'], row['lot'])
    folium.Marker(location, popup=row['s_name']).add_to(map_osm)

map_osm

TypeError: must be real number, not str

In [ ]:
df[['s_name', 'gugun_name', 'doro_address', 'tel']]

# 요약
* requests.post() 데이터 가져오기
* json_normalize(): JSON 데이터 DataFrame으로 전환
* folium 을 사용하여 지도에 위치 표시

### 2018 FinanceData.KR [fb.com/financedata](fb.com/financedata)